In [ ]:
# %load_ext autoreload
# %autoreload 2
import wandb
from torch.utils.data import DataLoader
from torch import optim
from clap import Clap
from clap.datasets import ClapDataset
from clap.training import ClapTrainer, create_scheduler, SymmetricCrossEntropyLoss
from clap.utils import get_target_device, load_clap_config, set_random_seed

# Train CLAP on audio captioning datasets AudioCaps and ClothoV2

In [ ]:
# Load config for audio processing and get target device
config_path = r"C:\Users\leon\Documents\ML_Projects\Custom-CLAP\clap\configs\clap_htsat-tiny_gpt2_v1.yml"
ckpt_path = r"C:\Users\leon\Documents\ML_Projects\Custom-CLAP\clap\checkpoints\clap_htsat-tiny_gpt2_v1.ckpt"
config = load_clap_config(config_path=config_path)
device = get_target_device()

In [ ]:
# Load Datasets
seed = set_random_seed(config["training"]["seed"])
dataset_paths = [r"C:\Users\leon\Documents\ML_Projects\Custom-CLAP\clap\datasets\audiocaps", r"C:\Users\leon\Documents\ML_Projects\Custom-CLAP\clap\datasets\clotho"]
train_dataset = ClapDataset(config=config, kinds=["train"], datasets=["AudioCaps", "Clotho"], datasets_paths=dataset_paths)
val_dataset = ClapDataset(config=config, kinds=["val"], datasets=["AudioCaps", "Clotho"], datasets_paths=dataset_paths)
test_dataset = ClapDataset(config=config, kinds=["test"], datasets=["AudioCaps", "Clotho"], datasets_paths=dataset_paths)

In [ ]:
# Use wandb logging (just skip and set enable_wandb_logging to False if not wanted)
wandb.login()

In [ ]:
wandb.init(
    # Set the wandb project where this run will be logged 
    project='CLAP-Training',
    name="First correct run",
    # Track hyperparameters
    config=config
)
config = wandb.config

In [ ]:
# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=config["training"]["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config["training"]["batch_size"])
test_loader = DataLoader(test_dataset, batch_size=config["training"]["batch_size"])

In [ ]:
# Define model, optimizer, scheduler and loss function
clap = Clap(config).to(device)
print(f"Number of parameters to train: {sum(p.numel() for p in clap.parameters())}")
optimizer = optim.Adam(clap.parameters(), lr=config["training"]["learning_rate"])
scheduler = create_scheduler(optimizer, warmup_steps=config["training"]["warmup_steps"], T_max=len(train_loader)*config["training"]["epochs"], milestones=config["training"]["milestones"])
loss_fn = SymmetricCrossEntropyLoss()
trainer = ClapTrainer(
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    model=clap,
    optimizer=optimizer,
    scheduler=scheduler,
    loss_fn=loss_fn,
    epochs=config["training"]["epochs"],
    enable_wandb_logging=False
)

In [ ]:
train_metrics, val_metrics, test_metrics = trainer.train_and_eval(ckpt_path, early_stopping=False)

In [ ]:
wandb.finish()